In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from battery_model import BatteryModel
from data import DataProvider
from GA import GeneticAlgorithm
from sklearn.metrics import r2_score
from scipy.optimize import least_squares


In [ ]:
battery_model = BatteryModel()

In [ ]:
data_provider = DataProvider()
df = data_provider.read_excel()

In [ ]:
df = data_provider.scale_data(df)
df.head()

In [ ]:
optimal_params = battery_model.get_optimal_ABCD_params()
A_opt, B_opt, C_opt, D_opt = optimal_params.x

print(f"Optimized C Value: {C_opt:.4f}")

plt.figure(figsize=(8, 5))
plt.scatter(battery_model.cycles, battery_model.capacity, label="Measured Data", color="red")
plt.plot(battery_model.cycles, battery_model.degradation_model(optimal_params.x, battery_model.cycles), label="Fitted Model", linestyle="--")
plt.xlabel("Cycle Count")
plt.ylabel("Remaining Capacity (%)")
plt.legend()
plt.title("Battery Degradation Model Fit")
plt.grid()
plt.show()

In [ ]:
def objective_function(params, df, C, isCharging):
    predicted_vals = []
    true_vals = []

    for _, row in df.iterrows():
        soc = row["SoC"]
        I = row["Battery Current(A)"]
        t = row["time_diff_sec"]

        predicted_v = battery_model.terminal_voltage(I, t, C, params, soc, isCharging)
        predicted_vals.append(predicted_v)
        true_vals.append(row["Battery Voltage(V)"])

    predicted_vals = np.array(predicted_vals)
    true_vals = np.array(true_vals)

    mse = np.mean((true_vals - predicted_vals) ** 2)

    #penalize error spread instead of raw prediction variance
    errors = true_vals - predicted_vals
    spread_penalty = np.var(errors)
    alpha = 0.5

    return mse + alpha * spread_penalty

In [ ]:
def plot(measured_df, predicted_df, isCharging):
    plt.figure(figsize=(15, 5))

    plt.plot(
        measured_df["SoC"], 
        measured_df["Battery Voltage(V)"], 
        marker='o', 
        linestyle='-', 
        label="Mean Voltage", 
        color='blue'
    )

    plt.plot(
        measured_df["SoC"], 
        predicted_df, 
        color='black',
        marker='o', 
        linestyle='-',
        label="Predicted Voltage", 
        alpha=0.5
    )

    plt.xlabel("State of Charge (SoC)")
    plt.ylabel("Voltage (V)")
    plt.title("Voltage vs. SoC")
    plt.legend()
    plt.grid(True)

    if isCharging == False:
        # Reverse the x-axis to display from 1 to 0
        plt.gca().invert_xaxis()

    plt.tight_layout()
    plt.show()

In [ ]:
charging_df = data_provider.get_charging_data(df)
charging_df.head(100)


In [ ]:
plt.figure(figsize=(8, 5))

plt.scatter(
        charging_df["SoC"], 
        charging_df["Battery Voltage(V)"], 
        marker='o',  
        color='blue'
    )
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
discharging_df = data_provider.get_discharging_data(df)
discharging_df.head(100)

In [ ]:
actual_voltages_charging = charging_df["Battery Voltage(V)"]
actual_voltages_discharging = discharging_df["Battery Voltage(V)"]

In [ ]:
initial_guess = np.array([
    6.05930148e-01, 1.89934042e-02, 6.34897039e-02, 5.80786935e+00,
    0.00000000e+00, 0.00000000e+00, 1.15731001e-03, 4.34165757e-02,
    2.84167937e-02, 2.83527549e-02, 5.62665077e+00, 0.00000000e+00,
    0.00000000e+00, 0.00000000e+00, 5.51815757e-01, 2.42309081e-02,
    7.38624760e-01, 8.56317765e-01, 1.55241002e-01, 4.10597812e-01,
    1.11276142e+00, 9.92462681e+00, 1.03530424e+01, 9.73174738e+00,
    4.95873543e-02, 1.09445309e+01, 4.15273779e+00, 9.70293078e-02,
    0.00000000e+00, 2.09624137e-01, 1.15631552e+01
])
# initial_guess = np.random.uniform(0.01, 1.0, 31)

In [ ]:
# Running LSM for charging

result_lsm = least_squares(objective_function, initial_guess, args=(charging_df, C_opt, True), bounds=(0, np.inf))
best_params_lsm_charging = result_lsm.x

print("Optimized Parameters (LSM):", best_params_lsm_charging)

In [ ]:
predicted_voltages_lsm_charging = battery_model.get_predicted_voltages(charging_df, C_opt, best_params_lsm_charging, isCharging=True)

In [ ]:
r2 = r2_score(actual_voltages_charging, predicted_voltages_lsm_charging)
print(f"Coefficient of Determination (R²): {r2:.4f}")

In [ ]:
plot(charging_df, predicted_voltages_lsm_charging, isCharging=True)

In [ ]:
# Running LSM for discharging
result_lsm = least_squares(objective_function, initial_guess, args=(discharging_df, C_opt, False), bounds=(0, np.inf))
best_params_lsm_discharging = result_lsm.x

print("Optimized Parameters (LSM):", best_params_lsm_discharging)

In [ ]:
predicted_voltages_lsm_discharging = battery_model.get_predicted_voltages(discharging_df, C_opt, best_params_lsm_discharging, isCharging=False)

In [ ]:
r2 = r2_score(actual_voltages_discharging, predicted_voltages_lsm_discharging)
print(f"Coefficient of Determination (R²): {r2:.4f}")

In [ ]:
plot(discharging_df, predicted_voltages_lsm_discharging, isCharging=False)

In [ ]:
# Running GA for charging
ga_optimizer = GeneticAlgorithm()
ga_optimizer.init_generation(best_params_lsm_charging)
best_params_ga = ga_optimizer.optimize(charging_df, C_opt, objective_function, isCharging=True)
print("Optimized Parameters (GA):", best_params_ga)

In [ ]:
predicted_voltages_ga_charging = battery_model.get_predicted_voltages(charging_df, C_opt, best_params_ga, isCharging=True)

In [ ]:
r2 = r2_score(actual_voltages_charging, predicted_voltages_ga_charging)
print(f"Coefficient of Determination (R²): {r2:.4f}")

In [ ]:
plot(charging_df, predicted_voltages_ga_charging, isCharging=True)

In [ ]:
# Running GA for discharging
ga_optimizer = GeneticAlgorithm()
ga_optimizer.init_generation(best_params_lsm_discharging)
best_params_ga_discharging = ga_optimizer.optimize(discharging_df, C_opt, objective_function, isCharging=False)
print("Optimized Parameters (GA):", best_params_ga_discharging)

In [ ]:
predicted_voltages_ga_discharging = battery_model.get_predicted_voltages(discharging_df, C_opt, best_params_ga_discharging, isCharging=False)

In [ ]:
r2 = r2_score(actual_voltages_discharging, predicted_voltages_ga_discharging)
print(f"Coefficient of Determination (R²): {r2:.4f}")

In [ ]:
plot(discharging_df, predicted_voltages_ga_discharging, isCharging=False)